# streamlined version

In [2]:
import sympy as sym
import math as m
import numpy as np
import scipy.optimize
from sympy import pprint
from scipy.optimize import fsolve
from scipy.optimize import least_squares
from scipy.optimize import minimize

In [3]:
w = m.e**((2/3)*m.pi*(1j))     # third root of unity
POVM_unnormalized = np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]])  # unnormalized POVM direction vectors
POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # normalized POVM direction vectors


In [4]:
# Rough

def qutrit_dot(qutrit1, qutrit2):
    q1 = np.array(qutrit1)
    q2 = np.array(qutrit2)
    return np.vdot(q1,q2)

# Another way, Creating a dictionary of pairs of Vector numbers and their inner product
# POVM_vec_np = (1/(2**0.5)) * np.array(POVM_unnormalized)
POVM_vec_np = np.array(POVM_unnormalized)
def qm_inner_product(vec1, vec2):
    return np.vdot(vec1, vec2)
inner_products = {}             # Compute the inner products and store in a dictionary
for i in range(len(POVM_vec_np)):
    for j in range(len(POVM_vec_np)):
        key = (i+1, j+1)
        value = qm_inner_product(POVM_vec_np[i], POVM_vec_np[j])
        if value.imag < 1e-14: 
            value = value.real 
            # value = round(value, 8)
        inner_products[key] = value 
# for key, value in inner_products.items():
#     print(f"Inner product of vectors {key}: {value}")
# verified it is symmertric, normalization holds etc.


In [80]:
#program used for finding the inner products in order to calculate the fourth element of each vector.
"""pair = (1,9)
print(f'inner product value associated to vectors: {pair} : ', inner_products[pair])
c4j = -inner_products[pair]/2           #analytical formula, see notebook
print('c4j:', c4j)"""

"pair = (1,9)\nprint(f'inner product value associated to vectors: {pair} : ', inner_products[pair])\nc4j = -inner_products[pair]/2           #analytical formula, see notebook\nprint('c4j:', c4j)"

In [81]:
#now finding the fifth element each by using the orthogonality condition of vector number 2 (which is known) and vectors j (number 3 and after)

c4j_list = [2,.5,.5,.5, (-.25-.433013j), -.25, .5, -.25, (-.25-.433013j)]           # fourth elements found earlier, not normalized at this point
# extending the 3d povm to 4d now that the fourth elements are known. this will be used for inner products and subsequently to find the fifth element.
four_lists = []                         # creating a list of the first four elements of each vector.
for i in range(9):
    # four_list = POVM_unnormalized[i].append(c4j_list[i])
    four_list = np.append(POVM_unnormalized[i], c4j_list[i])
    four_lists.append(four_list)


# storing possible pairs' inner products
four_inner_products = {}             
for i in range(len(four_lists)):
    for j in range(len(four_lists)):
        key = (i+1, j+1)
        value = np.vdot(four_lists[i], four_lists[j])
        if value.imag < 1e-14: 
            value = value.real 
            # value = round(value, 8)
        four_inner_products[key] = value

c5j_list = []                   # storing the fifth element for each vector after computing it.
for j in range(1,10):
    pair = (2,j)
    # print(f'four-inner product value associated to four_vectors: {pair} : ', four_inner_products[pair])
    c5j = -(2/np.sqrt(15))*four_inner_products[pair]           # see notebook
    # print(f'c5{pair[1]}:', c5j)
    c5j_list.append(c5j)

print('list of 5th entries (c52 is wrong, should be sqrt(15)/2): ',c5j_list)        # second entry is wrong here, careful

#correct list, Second entry corrected from -1.1618 to root(15)/2 (see notebook)
c5j_list = [-0.0, 1.936492 , 0.38729833462074165, (-0.38729833462074176-0.44721359549995776j), -0.19364916731037082, 0.5809475019311124, -0.38729833462074165, (-0.19364916731037093-0.4472135954999579j), 0.5809475019311124]


# creating List of first five entries, Used for creating full vector later
five_lists = []                         
for i in range(9):
    five_list = np.append(four_lists[i], c5j_list[i])
    five_lists.append(five_list)

# inner products of the 5d vectors, for normalization 
five_inner_products = {}
for i in range(len(five_lists)):
    key = (i+1, i+1)
    value = np.vdot(five_lists[i], five_lists[i])
    if value.imag < 1e-14: 
        value = value.real 
        # value = round(value, 12)
    five_inner_products[key] = value


print('Inner products of 5d vectors: ', five_inner_products)         


list of 5th entries (c52 is wrong, should be sqrt(15)/2):  [-0.0, -1.1618950038622249, 0.38729833462074165, (-0.38729833462074176-0.44721359549995776j), -0.1936491673103707, 0.5809475019311124, -0.38729833462074154, (-0.19364916731037093-0.4472135954999579j), 0.5809475019311124]
Inner products of 5d vectors:  {(1, 1): 6.0, (2, 2): 6.000001266064, (3, 3): 2.4, (4, 4): 2.5999999999999996, (5, 5): 2.287500258169, (6, 6): 2.4, (7, 7): 2.4, (8, 8): 2.3000000000000003, (9, 9): 2.587500258169}


In [82]:
# creating the abstract variables , to be added after the fifth entry for each of the nine vectors
# symbols = sym.symbols('c41 c51 c61 c71 c81 c42 c52 c62 c72 c82 c43 c53 c63 c73 c83 c44 c54 c64 c74 c84 c45 c55 c65 c75 c85 c46 c56 c66 c76 c86 c47 c57 c67 c77 c87 c48 c58 c68 c78 c88 c49 c59 c69 c79 c89')
symbols = sym.symbols('c61, c71, c81, c91, c62, c72, c82, c92, c63, c73, c83, c93, c64, c74, c84, c94, c65, c75, c85, c95, c66, c76, c86, c96, c67, c77, c87, c97, c68, c78, c88, c98, c69, c79, c89, c99')


First method of using free variables - 9th entry 0 identically for all the 9 vectors. SKIPPING FOR NOW..


Second method of using free variables - 9 variables used in first 2 vectors leaving us with 7 unknown vectors :


In [83]:
# Create the full 10D vectors v_i. Context: there are 9 free variables, we are setting the 9th enttry to be identically equal to 0 (tenth one is already zero). 
# There is another, probably better, method of cleverly using these 9 variables to actually reduce the number of equations. In that case, we use these 9 variables in the first 2 vectors, 5 and 4 
# respectively and then using these vectors we eliminate further coefficients or the unknown variables leaving us with actually only 21 equations(or something like that) .
NineDimVs_initial = []
for i in range(9):
    known_part = five_lists[i]            # Numerical entries from the five lists
    abstract_part = symbols[i*4:(i+1)*4]                         #  symbolic entries
    vector =  (1/sym.sqrt(6))*sym.Matrix(np.append(known_part, abstract_part)) 
    # NineDimVs.append(vector)
    NineDimVs_initial.append(vector.evalf())

NineDimVs_initial[0] = NineDimVs_initial[0].subs({'c61':0,'c71':0,'c81':0,'c91':0})     #manually changing the first vector and the second vector (The unknowns were all set to zero).
NineDimVs_initial[1] = NineDimVs_initial[1].subs({'c62':0,'c72':0,'c82':0,'c92':0})

u1_4, u2_4, u3_4, u4_4, u5_4, u6_4, u7_4, u8_4, u9_4 = sym.Matrix(NineDimVs_initial[0]),sym.Matrix(NineDimVs_initial[1]),sym.Matrix(NineDimVs_initial[2]),sym.Matrix(NineDimVs_initial[3]),sym.Matrix(NineDimVs_initial[4]),sym.Matrix(NineDimVs_initial[5]),sym.Matrix(NineDimVs_initial[6]),sym.Matrix(NineDimVs_initial[7]),sym.Matrix(NineDimVs_initial[8])
#above u vectors are the normalized vectors. the '_4' is Due to the fact that there are four unknown entries in each of those vectors (except the first two vectors which are fully known).

# pprint(np.sqrt(6)*u3_4) #eg (visualizing the unnormalized one, coz they are easier to understandMacBoook below 30% 🙂)
# pprint(sym.sqrt(6)*u4)



⎡       1.0       ⎤
⎢                 ⎥
⎢      -1.0       ⎥
⎢                 ⎥
⎢        0        ⎥
⎢                 ⎥
⎢       0.5       ⎥
⎢                 ⎥
⎢0.387298334620742⎥
⎢                 ⎥
⎢     1.0⋅c₆₃     ⎥
⎢                 ⎥
⎢     1.0⋅c₇₃     ⎥
⎢                 ⎥
⎢     1.0⋅c₈₃     ⎥
⎢                 ⎥
⎣     1.0⋅c₉₃     ⎦


Normalization entry, expressing the last entry as a function of the remaining variables.

In [84]:
"""# converting the last entry into normalization entry. Formula : |c9j| = sqrt(1- sum_i_1_to_8(|c_ij|^2)))

NineDimVs_4_variable = [u3_4,u4_4,u5_4,u6_4,u7_4,u8_4,u9_4]      #Again, first two vectors fully known, these remaining have four unknown variables. The last one of each is to be converted into normalization entry.
# NineDimVs_4_variable = [u3.evalf(),u4.evalf(),u5.evalf(),u6.evalf(),u7.evalf(),u8.evalf(),u9.evalf()]      #Again, first two vectors fully known, these remaining have four unknown variables. The last one of each is to be converted into normalization entry.

NineDimVs_3_variable = []
for i in NineDimVs_4_variable:
    u_j = i
    u_j[8] = sym.sqrt(1- sum([abs(i)**2 for i in u_j[:8]]))
    NineDimVs_3_variable.append(u_j)

#Finally, below we have the final version of the vectors. The first two are fully known, The remaining ones have first five entries known, then the next three variable, and the last one, the normalization entry.

u1, u2, u3, u4, u5, u6, u7, u8, u9 = u1_4, u2_4, NineDimVs_4_variable[0] , NineDimVs_4_variable[1], NineDimVs_4_variable[2], NineDimVs_4_variable[3], NineDimVs_4_variable[4], NineDimVs_4_variable[5] , NineDimVs_4_variable[6] 

# eg.
# pprint(u1)
# pprint((np.sqrt(6))*u3)
# pprint((np.sqrt(6))*u4)
# pprint(u4.subs({'c44':0,'c54':0,'c64':0,'c74':0}))"""

"# converting the last entry into normalization entry. Formula : |c9j| = sqrt(1- sum_i_1_to_8(|c_ij|^2)))\n\nNineDimVs_4_variable = [u3_4,u4_4,u5_4,u6_4,u7_4,u8_4,u9_4]      #Again, first two vectors fully known, these remaining have four unknown variables. The last one of each is to be converted into normalization entry.\n# NineDimVs_4_variable = [u3.evalf(),u4.evalf(),u5.evalf(),u6.evalf(),u7.evalf(),u8.evalf(),u9.evalf()]      #Again, first two vectors fully known, these remaining have four unknown variables. The last one of each is to be converted into normalization entry.\n\nNineDimVs_3_variable = []\nfor i in NineDimVs_4_variable:\n    u_j = i\n    u_j[8] = sym.sqrt(1- sum([abs(i)**2 for i in u_j[:8]]))\n    NineDimVs_3_variable.append(u_j)\n\n#Finally, below we have the final version of the vectors. The first two are fully known, The remaining ones have first five entries known, then the next three variable, and the last one, the normalization entry.\n\nu1, u2, u3, u4, u5, u6, u

In [85]:
# creating equations now amongst the last seven equations, all possible pairs, giving total 21 equations
# Not forgetting to take conjugates while doing inner product.
"""
# List of nine vectors
NineDimVecs = [u1, u2, u3, u4, u5, u6, u7, u8, u9]
Equations_ortho_dict = {}

# Create pairs of equations and store in dictionary
for i in range(len(NineDimVecs)):
    for j in range(i + 1, len(NineDimVecs)):
        key = f'E_{i+1}{j+1}'  # Create key as 'E_ij'
        # Equations_ortho_dict[key] = (i+1, j+1)  # Store pair (i+1, j+1) to reflect element positions
        expression = (NineDimVecs[i].conjugate().T*NineDimVecs[j])[0].evalf()           # Expression from inner product, final zero is to extract the inside of the matrix
        print(type(expression))
        if type(expression) == sym.core.numbers.Zero or type(expression) == sym.core.numbers.Float:
            if abs(expression) < 1e-10: 
                expression = 0
            else:
                expression = round(expression, 8)
            
        Equations_ortho_dict[key] = expression                    
        
for key, value in Equations_ortho_dict.items():
    print(f"Equation {key}: {value}")   """



# Assuming NineDimVecs is a list of NumPy arrays, if not, convert them first
# For example, if originally in SymPy: u1 = np.array([complex(sym.re(u1), sym.im(u1)) for u1 in sympy_vector])

# List of nine vectors
# NineDimVecs = [u1, u2, u3, u4, u5, u6, u7, u8, u9]                   # uncomment this line and comment the next one if using the last entry as normalization entry
NineDimVecs = [u1_4, u2_4, u3_4,u4_4,u5_4,u6_4,u7_4,u8_4,u9_4]      
Equations_ortho_dict = {}

# Create pairs of equations and store in dictionary
for i in range(len(NineDimVecs)):
    for j in range(i , len(NineDimVecs)):
        key = f'E_{i+1}{j+1}'  # Create key as 'E_ij'
        # Compute inner product using NumPy with conjugate
        expression = np.vdot(NineDimVecs[i], NineDimVecs[j])
        Equations_ortho_dict[key] = expression



# how to substitute values in the equations

# tt = Equations_ortho_dict['E_35']
# pprint(tt)
# # print('','\n \n \n \n \n \n ',)
# print(tt.subs({'c64':0,'c74':0,'c84':0, 'c63':0, 'c73':0, 'c83':0, 'c65': 0,'c75': 0, 'c85': 0}))
print(Equations_ortho_dict)

{'E_11': 1.00000000000000, 'E_12': 0, 'E_13': 0, 'E_14': -2.77555756156289e-17 + 5.55111512312578e-17*I, 'E_15': 4.16333634234434e-17 - 9.9369260264659e-8*I, 'E_16': -1.38777878078145e-17 - 0.144337567297406*I, 'E_17': -2.77555756156289e-17 + 5.55111512312578e-17*I, 'E_18': -1.38777878078145e-17 - 0.144337567297406*I, 'E_19': 4.16333634234434e-17 - 9.9369260264659e-8*I, 'E_22': 1.00000021101067, 'E_23': 2.11010648676346e-8, 'E_24': -2.11010648953902e-8 - 2.43654109666203e-8*I, 'E_25': -1.05505324754507e-8 - 0.180421983964073*I, 'E_26': 3.16515972875742e-8, 'E_27': -2.11010648953902e-8 - 0.144337567297406*I, 'E_28': -1.05505324615729e-8 - 2.43654109666203e-8*I, 'E_29': 3.16515972875742e-8 - 0.0360844166666667*I, 'E_33': 0.166666666666667*c63*conjugate(c63) + 0.166666666666667*c73*conjugate(c73) + 0.166666666666667*c83*conjugate(c83) + 0.166666666666667*c93*conjugate(c93) + 0.4, 'E_34': 0.166666666666667*c64*conjugate(c63) + 0.166666666666667*c74*conjugate(c73) + 0.166666666666667*c84*co

In [86]:
"""#Creating a function that returns equations upon input variables for the sake of using solver

def Equations_sub(vars):              # vars is the list of variables
    if len(vars) < 21:              # if the list is not of length 21, we will pad it with zeros
        while len(vars) < 21:
            vars.append(0)
    c63, c73, c83, c64, c74, c84, c65, c75, c85, c66, c76, c86, c67, c77, c87, c68, c78, c88, c69, c79, c89 = vars          # the 21 variables given as input
        
    
    eqn_order = ['E_34', 'E_35', 'E_36', 'E_37', 'E_38', 'E_39', 'E_45', 'E_46', 'E_47', 'E_48', 'E_49', 'E_56', 'E_57', 'E_58', 'E_59', 'E_67', 'E_68', 'E_69', 'E_78', 'E_79', 'E_89']

    substituted_eqns = []
    for i in eqn_order:
        eqn = Equations_ortho_dict[i]
        subbed_eqn = eqn.subs({'c63':c63, 'c73':c73, 'c83':c83, 'c64':c64, 'c74':c74, 'c84':c84, 'c65':c65, 'c75':c75, 'c85':c85, 'c66':c66, 'c76':c76, 'c86':c86, 'c67':c67, 'c77':c77, 'c87':c87, 'c68':c68, 'c78':c78, 'c88':c88, 'c69':c69, 'c79':c79, 'c89':c89})
        substituted_eqns.append(subbed_eqn.evalf())
    
    return substituted_eqns"""

"#Creating a function that returns equations upon input variables for the sake of using solver\n\ndef Equations_sub(vars):              # vars is the list of variables\n    if len(vars) < 21:              # if the list is not of length 21, we will pad it with zeros\n        while len(vars) < 21:\n            vars.append(0)\n    c63, c73, c83, c64, c74, c84, c65, c75, c85, c66, c76, c86, c67, c77, c87, c68, c78, c88, c69, c79, c89 = vars          # the 21 variables given as input\n        \n    \n    eqn_order = ['E_34', 'E_35', 'E_36', 'E_37', 'E_38', 'E_39', 'E_45', 'E_46', 'E_47', 'E_48', 'E_49', 'E_56', 'E_57', 'E_58', 'E_59', 'E_67', 'E_68', 'E_69', 'E_78', 'E_79', 'E_89']\n\n    substituted_eqns = []\n    for i in eqn_order:\n        eqn = Equations_ortho_dict[i]\n        subbed_eqn = eqn.subs({'c63':c63, 'c73':c73, 'c83':c83, 'c64':c64, 'c74':c74, 'c84':c84, 'c65':c65, 'c75':c75, 'c85':c85, 'c66':c66, 'c76':c76, 'c86':c86, 'c67':c67, 'c77':c77, 'c87':c87, 'c68':c68, 'c78':c78, 'c

In [87]:
"""# function that returns equation but adapted for working with complex numbers, total 21+21 variables

import numpy as np

def Equations_sub_complex(vars):                                # vars is the list of variables (42 in total, first 21 are magnitudes, remaining 21 are corresponding phases)
    if len(vars) < 42:  # Now we need 42 variables (21 magnitudes and 21 phases)
        while len(vars) < 42:
            vars.append(0)
    
    # first 21 are magnitudes and the next 21 are phases
    magnitudes = vars[:21]
    phases = vars[21:]

    # polar coordinates to complex numbers
    complex_vars = [r * np.exp(1j * theta) for r, theta in zip(magnitudes, phases)]
    
    # Unpack complex variables as needed for your equations
    c63, c73, c83, c64, c74, c84, c65, c75, c85, c66, c76, c86, c67, c77, c87, c68, c78, c88, c69, c79, c89 = complex_vars

    # Define or reference your equations here; this is a placeholder
    eqn_order = ['E_34', 'E_35', 'E_36', 'E_37', 'E_38', 'E_39', 'E_45', 'E_46', 'E_47', 'E_48', 'E_49', 'E_56', 'E_57', 'E_58', 'E_59', 'E_67', 'E_68', 'E_69', 'E_78', 'E_79', 'E_89']
    
    # Assuming Equations_ortho_dict is defined globally and contains the equations
    substituted_eqns = []
    for i in eqn_order:
        eqn = Equations_ortho_dict[i]
        # Substitute using a dictionary constructed from variable names and their values
        subbed_eqn = eqn.subs({'c63':c63, 'c73':c73, 'c83':c83, 'c64':c64, 'c74':c74, 'c84':c84, 'c65':c65, 'c75':c75, 'c85':c85, 'c66':c66, 'c76':c76, 'c86':c86, 'c67':c67, 'c77':c77, 'c87':c87, 'c68':c68, 'c78':c78, 'c88':c88, 'c69':c69, 'c79':c79, 'c89':c89}, simultaneous=True)
        substituted_eqns.append(subbed_eqn.evalf())
    
    residual_eqns = [abs(i.evalf())**2 for i in substituted_eqns]   # avoiding the problem of making both the real and imaginary parts zero individually. It works as long as the magnitudes/"residue" go to zero.

    output_residue = residual_eqns + residual_eqns  # for tricking the fsolve function to not throw errror "input length must match the output number of equations..". Doing this simply doubles the total residue.

    return output_residue

# Example usage
# initial_magnitudes = [1]*21  # Example initial magnitudes
# initial_phases = [0]*21     # Example initial phases (radians)
# initial_guesses = initial_magnitudes + initial_phases
# ig = initial_guesses
"""

'# function that returns equation but adapted for working with complex numbers, total 21+21 variables\n\nimport numpy as np\n\ndef Equations_sub_complex(vars):                                # vars is the list of variables (42 in total, first 21 are magnitudes, remaining 21 are corresponding phases)\n    if len(vars) < 42:  # Now we need 42 variables (21 magnitudes and 21 phases)\n        while len(vars) < 42:\n            vars.append(0)\n    \n    # first 21 are magnitudes and the next 21 are phases\n    magnitudes = vars[:21]\n    phases = vars[21:]\n\n    # polar coordinates to complex numbers\n    complex_vars = [r * np.exp(1j * theta) for r, theta in zip(magnitudes, phases)]\n    \n    # Unpack complex variables as needed for your equations\n    c63, c73, c83, c64, c74, c84, c65, c75, c85, c66, c76, c86, c67, c77, c87, c68, c78, c88, c69, c79, c89 = complex_vars\n\n    # Define or reference your equations here; this is a placeholder\n    eqn_order = [\'E_34\', \'E_35\', \'E_36\', 

In [88]:
# the previous code works but it is too slow one evaluation takes five seconds now trying hopefully quicker way by lambdifying the equations

# lambdifying for faster numerical evaluation. Below is lambdifyied dictionary of equations
## uncomment the below line and comment out the line afterwards in order to use the version with normalization entry
# equations_lambdas = {key: sym.lambdify(('c63', 'c73', 'c83', 'c64', 'c74', 'c84', 'c65', 'c75', 'c85', 'c66', 'c76', 'c86', 'c67', 'c77', 'c87', 'c68', 'c78', 'c88', 'c69', 'c79', 'c89'), eq, 'numpy') for key, eq in Equations_ortho_dict.items()}
equations_lambdas = {key: sym.lambdify(('c63', 'c73', 'c83', 'c93', 'c64', 'c74', 'c84', 'c94', 'c65', 'c75', 'c85', 'c95', 'c66', 'c76', 'c86', 'c96', 'c67', 'c77', 'c87', 'c97', 'c68', 'c78', 'c88', 'c98', 'c69', 'c79', 'c89', 'c99'), eq, 'numpy') for key, eq in Equations_ortho_dict.items()}



# equations_lambdas['E_34'](1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1)  # works

def Equations_ortho_obj_fn(vars):                                # vars is the list of variables (42 in total (56 if normalization entry is not in terms of other entries), first 21 are magnitudes, remaining 21 are corresponding phases)
    # if len(vars) < 42:  # in case less than 42 variables are given, we will pad it with zeros
    #     while len(vars) < 42:
    #         vars.append(0)
    
    # throw error if length of vars  is not even 
    if len(vars) % 2 != 0:
        raise ValueError('Number of variables must be even, check the input vars')

    magnitudes = vars[:int(len(vars)/2)]
    phases = vars[int(len(vars)/2):]
    complex_vars = [r * np.exp(1j * theta) for r, theta in zip(magnitudes, phases)]
    
    # Unpack complex variables as needed for your equations
    # c63, c73, c83, c64, c74, c84, c65, c75, c85, c66, c76, c86, c67, c77, c87, c68, c78, c88, c69, c79, c89 = complex_vars 
    c63, c73, c83, c93, c64, c74, c84, c94, c65, c75, c85, c95, c66, c76, c86, c96, c67, c77, c87, c97, c68, c78, c88, c98, c69, c79, c89, c99 = complex_vars 

    # Define or reference your equations here; this is a placeholder
    eqn_order = ['E_34', 'E_35', 'E_36', 'E_37', 'E_38', 'E_39', 'E_45', 'E_46', 'E_47', 'E_48', 'E_49', 'E_56', 'E_57', 'E_58', 'E_59', 'E_67', 'E_68', 'E_69', 'E_78', 'E_79', 'E_89']
    
    # Assuming Equations_ortho_dict is defined globally and contains the equations
    substituted_eqns = []
    for i in eqn_order:
        substituted_eqns.append(equations_lambdas[i](*complex_vars))
    
    residual_eqns = [abs(i)**2 for i in substituted_eqns]   # avoiding the problem of making both the real and imaginary parts zero individually. It works as long as the magnitudes/"residue" go to zero.
    output_residue = residual_eqns + residual_eqns  # for tricking the fsolve function to not throw errror "input length must match the output number of equations..". Doing this simply doubles the total residue.
    # return output_residue                         # , uncomment it and comment next when using fsolve instead of minimize
    return sum(output_residue)


In [89]:
#toy example of solving 10 nonlinaer equations

# Define the function that models the equations
def eqquations(variables):

    x1, x2, x3, x4, x5, x6, x7, x8, x9, x10 = variables
    eq1 = x1**2 + np.sin(x2) - x3 - 1
    eq2 = np.exp(x2) + x1 - x4**2
    eq3 = x3 + np.log(1 + x5**2) - x6 - 2
    eq4 = x4 + x7 - np.cos(x5)
    eq5 = x5**3 - x8 + x1 + 1
    eq6 = x6**2 - x9 + np.sin(x7) - 1
    eq7 = x7 + x10**2 - np.sqrt(x8 )
    eq8 = x8 + 2*x9 - np.exp(-x6) - 3
    eq9 = x9 + np.tan(x10) - x2**2
    eq10 = x10 - x1**2 + np.cos(x3) - 2
    return [eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8, eq9, eq10]


# Initial guess for the variables
rand_guess = [0.5, 0.5, 0.5, 0.5, .5, .7, .5, .5, .5, 1]
# initial_guess = [0.5, 0.5, 0.5, 0.5]

# Solve the system of equations


sol_list = []
for i in range(10):
    rand_guess = np.random.rand(10)
    sol = fsolve(eqquations, rand_guess)
    sol_list.append(sol)

fun_list = [sum([abs(j) for j in eqquations(i)]) for i in sol_list]
print('residual values', fun_list)
print('\n \n solutions', sol_list)

print('check', eqquations([-.04, -3.16154025e+00, -.978259822,  0, 1.52659862, -1.775136,  4.17515813,  4.51538863e+00, 2.19284861e+00,  1.44332699e+00]))
print('check', eqquations([-4.23545319e-02, -3.16154025e+00, -9.78259822e-01,  2.43174115e-03, 1.52659862e+00, -1.77513638e+00,  4.17515813e-02,  4.51538863e+00, 2.19284861e+00,  1.44332699e+00]))


residual values [0.9445697229408111, 10.31198362752018, 0.9009541104702086, 10.96712431930986, 11.22367047335062, 0.9316753612005708, 9.72733073472612, 12.514340334853015, 11.614347862143248, 11.556616470743545]

 
 solutions [array([-0.21520902,  1.12285791,  0.02409842, -1.67845901,  1.52093514,
       -0.69010971,  1.61150512,  4.28536588,  0.37459444,  0.75721629]), array([0.44346542, 0.33283264, 0.30875861, 0.84422173, 0.56555471,
       0.27627117, 0.14986877, 0.44609103, 0.14573282, 0.71093402]), array([-0.24034382,  1.15731068,  0.00864173, -1.70166956,  1.50343091,
       -0.76261863,  1.63434099,  4.14253293,  0.5131782 ,  0.73164069]), array([0.89821556, 0.67553169, 0.62474114, 0.68530074, 0.66146031,
       0.23760506, 0.09721935, 0.94207383, 0.00565996, 0.53263108]), array([0.02789894, 0.19142089, 0.11338514, 0.03187758, 0.4752125 ,
       0.70360163, 0.09319556, 0.19385105, 0.36147003, 0.25443613]), array([-0.20452725,  1.16320426,  0.01626578, -1.71092548,  1.50802867,
 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
/var/folders/z5/8__h5b611fz3d61c954t7_540000gp/T/ipykernel_3318/2184089955.py:13: RuntimeWarning: invalid value encountered in sqrt
  eq7 = x7 + x10**2 - np.sqrt(x8 )
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [55]:
# -writing the code now for the actual problem using minimize function but first writing constraints as individual functions
# constraints 
def normalization_vec3(vars):
    vec = 3
    eqn = f'E_{vec}{vec}'
    # split into magnitudes and phases
    magnitudes = vars[:28]
    phases = vars[28:]
    complex_vars = [r * np.exp(1j * theta) for r, theta in zip(magnitudes, phases)] 
    c63, c73, c83, c93, c64, c74, c84, c94, c65, c75, c85, c95, c66, c76, c86, c96, c67, c77, c87, c97, c68, c78, c88, c98, c69, c79, c89, c99 = complex_vars 
    residue = 1- equations_lambdas[eqn](*complex_vars)
    return residue

def normalization_vec4(vars):
    vec = 4
    eqn = f'E_{vec}{vec}'
    # split into magnitudes and phases
    magnitudes = vars[:28]
    phases = vars[28:]
    complex_vars = [r * np.exp(1j * theta) for r, theta in zip(magnitudes, phases)] 
    c63, c73, c83, c93, c64, c74, c84, c94, c65, c75, c85, c95, c66, c76, c86, c96, c67, c77, c87, c97, c68, c78, c88, c98, c69, c79, c89, c99 = complex_vars 
    residue = 1- equations_lambdas[eqn](*complex_vars)
    return residue

def normalization_vec5(vars):
    vec = 5
    eqn = f'E_{vec}{vec}'
    magnitudes = vars[:28]
    phases = vars[28:]
    complex_vars = [r * np.exp(1j * theta) for r, theta in zip(magnitudes, phases)] 
    c63, c73, c83, c93, c64, c74, c84, c94, c65, c75, c85, c95, c66, c76, c86, c96, c67, c77, c87, c97, c68, c78, c88, c98, c69, c79, c89, c99 = complex_vars 
    residue = 1- equations_lambdas[eqn](*complex_vars)
    return residue

def normalization_vec6(vars):
    vec = 6
    eqn = f'E_{vec}{vec}'
    magnitudes = vars[:28]
    phases = vars[28:]
    complex_vars = [r * np.exp(1j * theta) for r, theta in zip(magnitudes, phases)] 
    c63, c73, c83, c93, c64, c74, c84, c94, c65, c75, c85, c95, c66, c76, c86, c96, c67, c77, c87, c97, c68, c78, c88, c98, c69, c79, c89, c99 = complex_vars 
    residue = 1- equations_lambdas[eqn](*complex_vars)
    return residue

def normalization_vec7(vars):
    vec = 7
    eqn = f'E_{vec}{vec}'
    magnitudes = vars[:28]
    phases = vars[28:]
    complex_vars = [r * np.exp(1j * theta) for r, theta in zip(magnitudes, phases)] 
    c63, c73, c83, c93, c64, c74, c84, c94, c65, c75, c85, c95, c66, c76, c86, c96, c67, c77, c87, c97, c68, c78, c88, c98, c69, c79, c89, c99 = complex_vars 
    residue = 1- equations_lambdas[eqn](*complex_vars)
    return residue

def normalization_vec8(vars):
    vec = 8
    eqn = f'E_{vec}{vec}'
    magnitudes = vars[:28]
    phases = vars[28:]
    complex_vars = [r * np.exp(1j * theta) for r, theta in zip(magnitudes, phases)] 
    c63, c73, c83, c93, c64, c74, c84, c94, c65, c75, c85, c95, c66, c76, c86, c96, c67, c77, c87, c97, c68, c78, c88, c98, c69, c79, c89, c99 = complex_vars 
    residue = 1- equations_lambdas[eqn](*complex_vars)
    return residue

def normalization_vec9(vars):
    vec = 9
    eqn = f'E_{vec}{vec}'
    magnitudes = vars[:28]
    phases = vars[28:]
    complex_vars = [r * np.exp(1j * theta) for r, theta in zip(magnitudes, phases)] 
    c63, c73, c83, c93, c64, c74, c84, c94, c65, c75, c85, c95, c66, c76, c86, c96, c67, c77, c87, c97, c68, c78, c88, c98, c69, c79, c89, c99 = complex_vars 
    residue = 1- equations_lambdas[eqn](*complex_vars)
    return residue
# above are the constraints for the normalization equations. 


constraints = [{'type': 'eq', 'fun': normalization_vec3}, {'type': 'eq', 'fun': normalization_vec4}, {'type': 'eq', 'fun': normalization_vec5}, {'type': 'eq', 'fun': normalization_vec6}, {'type': 'eq', 'fun': normalization_vec7}, {'type': 'eq', 'fun': normalization_vec8}, {'type': 'eq', 'fun': normalization_vec9}] 

# solving the problem now using minimize

initial_guess = [1]*28 + [0]*28
sol = minimize(Equations_ortho_obj_fn, initial_guess, method='SLSQP', constraints=constraints)
sol


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.07371755105882578
       x: [ 1.629e+00 -5.360e-01 ...  1.140e+00  1.489e+00]
     nit: 37
     jac: [ 3.960e-02 -1.290e-02 ...  1.070e-06 -6.208e-06]
    nfev: 2115
    njev: 37

In [90]:
#write a for loop for creating random points in the parameter space and running the minimize function from those and then picking out the samllest solutions.
sol_list = []
params_list = []    
for i in range(100):
    rand_guess = np.concatenate((np.random.rand(28)*2, np.random.rand(28)*2*m.pi))
    sol = minimize(Equations_ortho_obj_fn, rand_guess, method='SLSQP', constraints=constraints)
    sol_list.append(sol.fun)
    params_list.append(sol.x)
    
#zip and sort, pick the smallest ten
sol_params = list(zip(sol_list, params_list))
sol_params.sort()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_numdiff.py:598: ComplexWarning: Casting complex values to real discards the imaginary part
  J_transposed[i] = df / dx
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_slsqp_py.py:422: ComplexWarning: Casting complex values to real discards the imaginary part
  slsqp(m, meq, x, xl, xu, fx, c, g, a, acc, majiter, mode, w, jw,


In [91]:
print('solutions', sorted(sol_list))

solutions [0.07371723114819353, 0.0737172601899207, 0.07371726453331223, 0.07371727416407045, 0.07371728067472118, 0.07371729008621591, 0.07371729807794192, 0.07371731661885703, 0.07371733115763283, 0.07371734843446966, 0.0737173519924838, 0.07371735819362953, 0.07371737022469901, 0.07371737099920328, 0.07371737822487963, 0.07371740688052991, 0.07371740985830308, 0.07371741142377353, 0.07371741626444699, 0.07371742225818839, 0.07371742536284245, 0.07371742678223686, 0.07371743196209085, 0.0737174359656057, 0.07371745298581632, 0.07371746379129279, 0.07371747669784794, 0.07371747911634609, 0.07371749252916422, 0.07371749604199661, 0.07371750935730154, 0.07371752353085026, 0.07371753633841929, 0.07371754875820372, 0.07371755754267277, 0.07371758276449394, 0.07371758457342484, 0.07371758763738873, 0.07371758765033538, 0.07371758861163927, 0.07371760497946249, 0.07371760789910344, 0.07371761162228521, 0.07371761844094943, 0.07371762082135325, 0.07371762094705471, 0.07371763206914259, 0.073

In [92]:
(np.arccos(np.sqrt(.073/42)))*(180/m.pi) 

87.61061955497705

In [24]:
# solving the actual problem using fsolve

sol_list = []
fun_list = []
# Bounds for the variables, comes from normalization consideration. see abouve, in a markdown cell
lower_bounds = [0] * 21 + [0] * 21  # Lower bounds for magnitudes and phases
upper_bounds = [2] * 21 + [2 * np.pi] * 21  # Upper bounds for magnitudes and phases

for i in range(1000):
    rand_guess = np.random.rand(42)
    rand_guess[21:] = rand_guess[21:]*2*np.pi
    sol = fsolve(Equations_ortho_obj_fn, rand_guess)
    fun = sum(Equations_ortho_obj_fn(sol))
    if fun < 4:
        sol_list.append(sol)
        fun_list.append(fun)

print(' small residue values : \n ', sorted(fun_list))

# func_list = [sum(Equations_sub_complex(i)) for i in sol_list]         # list of residues for each set of optimized parameters/variables
# print('\n \n optimized solutions : \n ', sol_list)


 small residue values : 
  []


In [125]:
# a million trials results
print('residue values a million trials : \n ', fun_list)
print('solutions a million trials : \n ', sol_list)

residue values a million trials : 
[3.982035224274555, 3.8359810854813614, 3.7127399382999964, 3.9391902724353947, 3.974282296368057, 3.8611724099909135, 3.932231371842751, 3.848579281279552, 3.6514649395042165, 3.9785266805465302, 3.7803485853315624, 3.9415584716434697, 3.651764228206487, 3.8996365059023455, 3.946535047341419, 3.9495872545375854, 3.8339851389992767, 3.9991041258350593, 3.874999422227794]
solutions a million trials: 
[array([0.97298991, 0.51672579, 0.46529802, 0.71886992, 0.96100051, 0.98670244, 0.05313418, 0.98400582, 0.36702111, 0.72012038, 0.49734017, 0.76417935, 0.46043416, 0.90474995, 0.69244048, 0.96213224, 0.71525187, 0.92021343, 0.93551432, 0.90224976, 0.07297566, 1.54023706, 3.585578  , 2.25917983, 5.06114398, 3.82691407, 5.8461731 , 1.5860423 , 0.41520867, 5.71437375, 4.75685429, 4.87045908, 5.13665565, 4.29464707, 3.45908974, 1.70506076, 6.01449216, 1.81070582, 2.94075217, 1.54073878, 5.42910517, 3.28298972]), 
array([0.988469  , 0.9117535 , 0.81129581, 0.71609331, 0.94481021,0.51751697, 0.95258264, 0.76182202, 0.8793652 , 0.03645617,0.90314746, 0.237     , 0.0946929 , 0.77683647, 0.87264398,0.61469068, 0.96495139, 0.95236755, 0.98744774, 0.53382437,0.64213816, 4.53280559, 1.68510939, 3.81108981, 0.62599356,3.21580034, 0.76557187, 3.25183847, 2.02790398, 4.02643068,3.72056474, 0.42853255, 1.57808436, 1.25487107, 5.36032881,1.06100633, 3.98411975, 2.56145317, 0.6354886 , 3.54922227,3.77702863, 6.21207483]), 
array([0.81829329, 0.93567711, 0.12249826, 0.90164832, 0.68487115,0.94138326, 0.73559334, 0.06649686, 0.35976823, 0.90934007,0.87201197, 0.91645238, 0.80970729, 0.76995194, 0.97334007,0.09425564, 0.9957041 , 0.50744389, 0.35844987, 0.86569977,0.91133583, 3.07215116, 1.10758455, 3.14538475, 6.08914427,2.38667382, 4.64697302, 0.64076491, 1.87553167, 1.01433977,2.69407046, 5.49508296, 3.9525332 , 4.70034608, 3.70078335,6.07827703, 1.93424831, 6.26709405, 2.50782905, 4.72445918,3.17426983, 3.29488231]), 
array([0.87497275, 0.62205439, 0.76479024, 0.74794498, 0.5065089 ,0.77312852, 0.41709392, 0.88653975, 0.90680954, 0.90833929,0.80513608, 0.22180933, 0.93615215, 0.79694615, 0.44916213,0.21106562, 0.52106188, 0.45681396, 0.88045791, 0.93865875,0.8058492 , 1.06384415, 4.91248913, 1.89461817, 2.50922313,2.85103986, 3.84680893, 0.05950288, 1.81258602, 4.24740728,3.1887232 , 2.36329574, 1.1370655 , 5.84281707, 5.79080162,0.07798779, 0.39968536, 4.74553528, 2.03342379, 1.22474043,5.16844295, 4.5417061 ]), 
array([0.81554543, 0.50388969, 0.2529717 , 0.22794871, 0.55239955,0.96140362, 0.8828843 , 0.90744989, 0.84035907, 0.65633245,0.95464039, 0.73956509, 0.87350375, 0.05674168, 0.84739046,0.81071033, 0.94709788, 0.69746435, 0.04594255, 0.93501866,0.89344742, 3.11044199, 5.98193016, 4.74561774, 2.57860561,3.85212687, 2.75947104, 3.8215062 , 1.81816619, 0.71215226,5.43101533, 4.61815443, 2.71813212, 0.59953354, 3.96772544,5.79988724, 0.69798173, 4.00531168, 5.65458598, 1.17835275,2.65721875, 4.94208782]), 
array([0.99665242, 0.77609754, 0.50672665, 0.6441524 , 0.61448123,0.97714922, 0.932925  , 0.66058143, 0.74078167, 0.48970889,0.23752155, 0.96880306, 0.85528107, 0.82213908, 0.99669662,0.03194129, 0.90676416, 0.71244764, 0.9933681 , 0.94282404,0.8852    , 0.22726507, 2.72133158, 0.61566023, 1.7460757 ,1.12420706, 2.99912638, 5.2120737 , 3.04366926, 1.55698507,1.97697855, 0.65717565, 5.69893371, 3.17446148, 4.6119381 ,0.87416178, 4.8092023 , 4.3985984 , 3.88049223, 1.73751396,0.10102187, 3.56042696]), 
array([0.22005731, 0.64709722, 0.76716811, 0.96092818, 0.07210938,0.77973793, 0.98605575, 0.87548307, 0.82836508, 0.95875014,0.92434048, 0.92722106, 0.64060636, 0.93628213, 0.91722907,0.78559181, 0.40649433, 0.74491495, 0.66202379, 0.93589335,0.70804504, 5.20049768, 6.23825047, 3.63189743, 1.53282544,3.34578619, 5.65456633, 3.0812603 , 4.93476254, 3.3481003 ,0.36164795, 2.23547625, 1.43753244, 4.32677638, 4.79416011,1.16870273, 4.84831039, 2.50771876, 5.54870221, 3.71227854,5.21361121, 2.03814402]), 
array([0.76749054, 0.97170543, 0.29299768, 0.85826047, 0.88875533,0.88907865, 0.92815908, 0.79994434, 0.90027174, 0.9431542 ,0.92104522, 0.78039901, 0.71183678, 0.99347802, 0.95329477,0.89034419, 0.59917745, 0.30612141, 0.06880688, 0.50010456,0.81808794, 5.09768283, 3.03350951, 0.02417551, 0.36145466,2.35287078, 3.56145598, 1.01081781, 5.57023477, 2.61226022,3.29660186, 3.17120513, 4.29804803, 5.51774473, 5.4895324 ,0.12583211, 3.19946167, 5.75508387, 1.07504925, 3.84782926,0.97778787, 5.39628971]), 
array([0.8854067 , 0.71268933, 0.47490448, 0.39515183, 0.99598057,0.71911237, 0.86537505, 0.71081116, 0.84255513, 0.73391906,0.62839301, 0.91220679, 0.85875705, 0.71938648, 0.69633999,0.91589999, 0.94817622, 0.95979522, 0.96705582, 0.30676695,0.81602543, 3.58638187, 4.1552982 , 0.562191  , 4.64616326,4.37940289, 1.32136702, 0.26645994, 2.07873178, 2.25163746,0.69845332, 3.86985454, 4.21856106, 1.29090961, 1.79256196,5.92198266, 3.48148007, 1.79644903, 3.88591402, 5.44115906,5.74698365, 2.81542804]), 
array([6.24363150e-01, 7.36355587e-01, 7.26535906e-01, 9.77850318e-01,9.89607756e-01, 5.82188031e-01, 9.64759616e-01, 4.13724681e-01,8.17908212e-01, 4.94040027e-03, 7.22333065e-01, 9.92535953e-01,9.41440439e-01, 3.16991976e-01, 5.11314323e-01, 6.02898965e-01,5.25510430e-01, 9.90262299e-01, 9.47571722e-01, 9.79973552e-02,7.87705857e-01, 6.27215040e+00, 3.54378841e+00, 2.48311472e+00,6.25531464e-01, 4.66427815e-01, 5.86122393e+00, 1.97912249e+00,1.39556193e+00, 3.75651202e+00, 2.36958933e+00, 1.05131497e+00,6.15164864e+00, 4.82461873e+00, 1.31153497e+00, 3.94514543e+00, 5.17603804e+00, 3.13147335e+00, 5.67324353e+00, 2.90982148e+00, 2.66125547e+00, 4.75278486e+00]), 
array([0.80270658, 0.98377205, 0.95676745, 0.95942306, 0.67613408,0.8598829 , 0.33986919, 0.90195971, 0.04089026, 0.29435698,0.92757442, 0.54781089, 0.6473289 , 0.49859457, 0.95167745,0.4474728 , 0.70935707, 0.49343057, 0.59299547, 0.57708415,0.8272441 , 4.6081501 , 1.4670126 , 6.11923087, 2.86770137,3.59931429, 2.3249655 , 3.70998605, 1.70136648, 4.19244971,2.75201501, 0.28761216, 0.51903795, 1.81321741, 5.94130103,2.93278531, 6.10583429, 6.01527515, 5.73565999, 6.1377817 ,3.79988075, 3.82530809]), 
array([0.83348143, 0.36386011, 0.50183989, 0.7138289 , 0.62161382,0.8412999 , 0.96747999, 0.88111885, 0.48716662, 0.51619441,0.71576864, 0.75631056, 0.98904838, 0.96886933, 0.94815954,0.68975372, 0.06803152, 0.65179637, 0.54620152, 0.79833584,0.92227289, 4.85762889, 4.54336861, 5.75471971, 0.01886638,6.28007148, 3.99878555, 5.27066326, 2.06542504, 0.80448519,5.36558587, 2.36180465, 4.18201354, 2.43654289, 0.06591708,1.63265472, 2.36878349, 4.36588822, 2.15008801, 1.38289964,4.42691757, 2.45147433]), 
array([0.80005756, 0.93898763, 0.91150082, 0.98913639, 0.72073724,0.9054846 , 0.81648453, 0.81656313, 0.59929659, 0.52044822,0.68526216, 0.5162425 , 0.14738875, 0.79929697, 0.47255195,0.94228129, 0.27867774, 0.81790803, 0.97225026, 0.87903766,0.46065479, 1.70241918, 0.67408196, 3.23522289, 0.76845981,3.38450321, 1.61512523, 1.77992257, 6.27364722, 2.14010043,5.4459876 , 1.64225549, 1.71042868, 4.72569261, 4.23334382,4.92866359, 3.16859536, 4.60757981, 4.6964937 , 3.89140513,4.00844996, 5.82987879]), 
array([0.87859162, 0.83677152, 0.9520245 , 0.67563192, 0.94643141,0.20292701, 0.16405937, 0.90058634, 0.90657163, 0.98665351,0.81056292, 0.03069412, 0.66884241, 0.98781804, 0.91296503,0.37261898, 0.72147442, 0.31985561, 0.97370756, 0.83308905,0.95341341, 5.80723663, 4.01627514, 0.31485769, 1.23632803,2.08808413, 5.96853577, 1.53689119, 3.82610463, 0.33203212,5.36196036, 5.43639527, 5.70589271, 4.08089539, 1.26360397,5.04650561, 6.0651299 , 0.4652517 , 6.05946806, 3.88716534,4.54265373, 3.4989678 ]), 
array([0.8510679 , 0.45789622, 0.90984667, 0.11781314, 0.74207737,0.91582694, 0.05172128, 0.77910366, 0.72280599, 0.95777234,0.915781  , 0.97389074, 0.81579399, 0.99540496, 0.2466054 ,0.23486038, 0.80773316, 0.48461953, 0.55355521, 0.48491224,0.66422809, 3.96385208, 3.41172655, 5.71099286, 2.45025296,0.32512563, 0.67095815, 4.31992967, 5.89471426, 1.11348773,2.49013143, 5.10952365, 3.18129293, 1.32002343, 0.76519329,3.30370585, 0.38683389, 3.15892882, 4.85179086, 0.05290055,3.37608406, 2.33229519]), 
array([0.73386827, 0.92940319, 0.94775783, 0.94926259, 0.71339525,0.41692039, 0.8664301 , 0.50150075, 0.29601133, 0.38703766,0.97479225, 0.72329177, 0.57281285, 0.92364367, 0.78988938,0.69523927, 0.10614133, 0.54303594, 0.97017459, 0.74568161,0.64827664, 3.80678878, 4.44988348, 4.78073979, 0.97087094,4.94002045, 1.05640873, 3.56910365, 4.85722344, 5.83234706,3.23829184, 5.47332392, 3.58899925, 5.69631903, 1.74920952,1.68791298, 6.14779112, 1.92741172, 5.36834216, 1.88881542,0.99337524, 0.50270591]), 
array([0.47133813, 0.88879204, 0.81982039, 0.79537955, 0.32210912,0.90393419, 0.36307099, 0.64140534, 0.89494567, 0.75345366,0.96530753, 0.90410326, 0.9147269 , 0.84477917, 0.77163662,0.97492792, 0.10845905, 0.57909297, 0.69549621, 0.90724595,0.91123736, 2.94736359, 3.08056323, 1.26636522, 2.00961028,3.41751581, 5.71327725, 2.97680751, 5.84821385, 0.9062879 ,2.66113646, 6.04870401, 4.5516475 , 5.00276195, 2.3917072 ,2.44676526, 4.95344603, 2.53286172, 4.8674069 , 0.05416466,0.01917528, 3.60918525]), 
array([0.85517704, 0.77893215, 0.62552157, 0.90688669, 0.87419756,0.65202026, 0.39018462, 0.50450762, 0.21046564, 0.99588594,0.92331   , 0.97538724, 0.32140528, 0.69599593, 0.93139212,0.3809899 , 0.94749933, 0.00779921, 0.94177545, 0.99761331,0.72774264, 3.72430328, 2.26200836, 4.40379693, 3.05815189,6.04276824, 2.24954488, 4.44512277, 4.54234635, 1.06625065,2.15996311, 5.51777108, 6.24502708, 0.0211889 , 1.03648095,4.9348487 , 2.98382882, 3.4789297 , 1.19649161, 6.16370872,2.40638888, 0.60150551]), 
array([0.55707516, 0.44242589, 0.39886475, 0.93913357, 0.33316355,0.97536974, 0.73090977, 0.25974185, 0.05813651, 0.96079426,0.84776325, 0.96995089, 0.81972855, 0.89622903, 0.97282073,0.9073535 , 0.33104538, 0.75543836, 0.95698374, 0.90467385,0.06747931, 5.97849293, 1.3287365 , 2.92596361, 1.24301457,0.28033354, 1.98188449, 4.83240694, 1.18203227, 5.8019126 ,5.18123033, 5.34548137, 3.08563598, 2.12912186, 4.34274452,5.23647033, 4.68661472, 0.07745715, 5.58401433, 2.02607948,3.89107291, 4.21264256])]


residue values a million trials : 
  [3.982035224274555, 3.8359810854813614, 3.7127399382999964, 3.9391902724353947, 3.974282296368057, 3.8611724099909135, 3.932231371842751, 3.848579281279552, 3.6514649395042165, 3.9785266805465302, 3.7803485853315624, 3.9415584716434697, 3.651764228206487, 3.8996365059023455, 3.946535047341419, 3.9495872545375854, 3.8339851389992767, 3.9991041258350593, 3.874999422227794]
solutions a million trials : 
  [array([0.97298991, 0.51672579, 0.46529802, 0.71886992, 0.96100051,
       0.98670244, 0.05313418, 0.98400582, 0.36702111, 0.72012038,
       0.49734017, 0.76417935, 0.46043416, 0.90474995, 0.69244048,
       0.96213224, 0.71525187, 0.92021343, 0.93551432, 0.90224976,
       0.07297566, 1.54023706, 3.585578  , 2.25917983, 5.06114398,
       3.82691407, 5.8461731 , 1.5860423 , 0.41520867, 5.71437375,
       4.75685429, 4.87045908, 5.13665565, 4.29464707, 3.45908974,
       1.70506076, 6.01449216, 1.81070582, 2.94075217, 1.54073878,
       5.42910517, 3

In [121]:
print(fun_list)
sol_list

ig1 = [0.84831059, 0.93769384, 0.95831843, 0.58859621, 0.78698391, 0.01617136, 0.38021036, 0.74357347, 0.89590989, 0.91384242, 0.79239096, 0.73088171, 0.99233128, 0.630825  , 0.37716664, 0.66417887, 0.92989896, 0.77456388, 0.93153411, 0.23269057, 0.74926933, 1.33973103, 2.28439978, 2.22430362, 2.38278931, 2.94219745, 1.15914577, 6.01838602, 3.38553601, 0.86655198, 0.97912109, 0.8747625 , 0.44176277, 5.33628305, 0.03451751, 3.22213894, 0.10679886, 6.13340598, 0.16663588, 6.28072979, 5.81580979, 3.03451796]
ig2 = [0.92577424, 0.53177561, 0.71045436, 0.25267221, 0.65331493, 0.84376378, 0.91487707, 0.90737266, 0.03250126, 0.4035517 , 0.81886947, 0.47786241, 0.67270315, 0.47559299, 0.94256656, 0.94492701, 0.94986377, 0.81935677, 0.61447397, 0.39757651, 0.74926135, 4.1592364 , 0.00835869, 4.23720644, 2.80644293, 3.01239237, 3.57130737, 0.26413028, 5.13187599, 0.84238074, 0.85717772, 3.45012904, 0.81278649, 4.22967385, 4.14038335, 1.59026947, 2.28802455, 2.79934562, 5.70374577, 5.3460091 , 3.39321309, 0.67248036]
ig3 = [0.90907938, 0.64100869, 0.04848268, 0.84229242, 0.1561698 , 0.95481804, 0.15822135, 0.43900882, 0.42276214, 0.48541024, 0.84908305, 0.92533351, 0.46550131, 0.9822555 , 0.4140465 , 0.96723563, 0.51955532, 0.98685504, 0.94630788, 0.20265642, 0.70620038, 4.83580551, 5.79544151, 5.05866788, 0.29939708, 2.72087968, 0.75981019, 1.16572167, 2.26255085, 3.73442491, 2.90421789, 5.47926318, 6.14810024, 5.27496706, 4.95467262, 2.18237715, 4.68519714, 4.19704465, 4.23267254, 2.09015893, 4.37450751, 2.84155016]

[4.93861094589849, 4.986147789549319, 4.84321265341596, 4.870144188487684]


In [123]:
for i in [ig1, ig2, ig3]:
    # add small perturbations to the initial guess
    new_ig = [j + (1/2)*np.random.rand() for j in i]
    print(' previous residue:', sum(Equations_ortho_obj_fn(i)))
    sol = fsolve(Equations_ortho_obj_fn, new_ig)
    print(' optimized residue:', sum(Equations_ortho_obj_fn(sol)))

 previous residue: 4.9386109576373425
 optimized residue: nan
 previous residue: 4.986147793110423
 optimized residue: 3.8065600763957557
 previous residue: 4.843212664319748
 optimized residue: nan


<lambdifygenerated-448>:2: RuntimeWarning: invalid value encountered in sqrt
  return 0.166666666666667*c64*conjugate(c63) + 0.166666666666667*c74*conjugate(c73) + 0.166666666666667*c84*conjugate(c83) + 0.58309518948453*sqrt(-0.294117647058823*abs(c64)**2 - 0.294117647058823*abs(c74)**2 - 0.294117647058823*abs(c84)**2 + 1)*conjugate(sqrt(-0.277777777777778*abs(c63)**2 - 0.277777777777778*abs(c73)**2 - 0.277777777777778*abs(c83)**2 + 1)) + 0.1 - 0.173205080756888*1j
<lambdifygenerated-449>:2: RuntimeWarning: invalid value encountered in sqrt
  return 0.166666666666667*c65*conjugate(c63) + 0.166666666666667*c75*conjugate(c73) + 0.166666666666667*c85*conjugate(c83) + 0.609302859162092*sqrt(-0.269360288091711*abs(c65)**2 - 0.269360288091711*abs(c75)**2 - 0.269360288091711*abs(c85)**2 + 1)*conjugate(sqrt(-0.277777777777778*abs(c63)**2 - 0.277777777777778*abs(c73)**2 - 0.277777777777778*abs(c83)**2 + 1)) - 0.2 - 0.0360844166666667*1j
<lambdifygenerated-450>:2: RuntimeWarning: invalid value e

since the inner products of the five lists with themselves are never above 2.6 let's say and since the total normalization gives the value of the inner product with itself to be 6 therefore the mod squares of the remaining variables when summed cannot exceed 4 (3.6 to be precise). Thus , We have a bound on the magnitude part of each variable, that its mod square should be lesser than 4 ie. the variable should be smaller than 2. 
|c_ij| < 2, for every variable of the 21 variables.

In [92]:
sol_list

[array([0.1443033 , 0.26772106, 0.31732649, 0.54410489, 0.12217749,
        0.32240162, 0.89090404, 0.09263855, 0.66142615, 0.78043501,
        0.49529279, 0.76920639, 0.92177663, 0.57713516, 0.73608335,
        0.85384013, 0.81264459, 0.63794865, 0.67100258, 0.86999407,
        0.89036437, 1.00365224, 6.26584684, 4.62567647, 1.24713735,
        4.05258642, 3.72526848, 3.71861478, 0.72378626, 1.26095229,
        3.11189595, 5.63269563, 2.92070023, 2.02199866, 0.27674475,
        5.56122125, 6.18189133, 4.61038131, 0.11495757, 4.35611706,
        3.72686667, 0.53880268]),
 array([0.51722186, 0.9305965 , 0.24627395, 0.5314911 , 0.92119117,
        0.17629173, 0.86416048, 0.44334979, 0.36081368, 0.88819072,
        0.18788448, 0.23331713, 0.9131788 , 0.68556323, 0.96037173,
        0.86794949, 0.88229804, 0.21140423, 0.91687901, 0.07941232,
        0.08050614, 0.26932547, 6.21752855, 4.4999163 , 4.60936732,
        5.0704888 , 2.96692341, 5.67695406, 2.91221748, 5.70125739,
        4.8540

In [ ]:
"""
# Define the equations based on the dot products
def equations(vars):
    # Unpack the variables
    c33, c34, c35, c43, c44, c45, c53, c54, c55, c63, c64, c65, c73, c74, c75, c83, c84, c85, c93, c94, c95 = vars
    # Compute the values of c36, c46, ... c96 based on the constraint of unit vectors
    c36 = np.sqrt(1 - abs(c33)**2 - abs(c34)**2 - abs(c35)**2)
    c46 = np.sqrt(1 - abs(c43)**2 - abs(c44)**2 - abs(c45)**2)
    c56 = np.sqrt(1 - abs(c53)**2 - abs(c54)**2 - abs(c55)**2)
    c66 = np.sqrt(1 - abs(c63)**2 - abs(c64)**2 - abs(c65)**2)
    c76 = np.sqrt(1 - abs(c73)**2 - abs(c74)**2 - abs(c75)**2)
    c86 = np.sqrt(1 - abs(c83)**2 - abs(c84)**2 - abs(c85)**2)
    c96 = np.sqrt(1 - abs(c93)**2 - abs(c94)**2 - abs(c95)**2)

    # Define the equations based on the dot product conditions
    eq1 = c33*c43 + c34*c44 + c35*c45 + c36*c46 + 1/10 - 1/4
    eq2 = c33*c53 + c34*c54 + c35*c55 + c36*c56 + 1/10 - 1/4
    eq3 = c33*c63 + c34*c64 + c35*c65 + c36*c66 + 1/10 - 1/4
    eq4 = c33*c73 + c34*c74 + c35*c75 + c36*c76 + 1/10 - 1/4
    eq5 = c33*c83 + c34*c84 + c35*c85 + c36*c86 + 1/10 - 1/4
    eq6 = c33*c93 + c34*c94 + c35*c95 + c36*c96 + 1/10 - 1/4
    eq7 = c43*c53 + c44*c54 + c45*c55 + c46*c56 + 1/10 - 1/4
    eq8 = c43*c63 + c44*c64 + c45*c65 + c46*c66 + 1/10 - 1/4
    eq9 = c43*c73 + c44*c74 + c45*c75 + c46*c76 + 1/10 - 1/4
    eq10 = c43*c83 + c44*c84 + c45*c85 + c46*c86 + 1/10 - 1/4
    eq11 = c43*c93 + c44*c94 + c45*c95 + c46*c96 + 1/10 - 1/4
    eq12 = c53*c63 + c54*c64 + c55*c65 + c56*c66 + 1/10 - 1/4
    eq13 = c53*c73 + c54*c74 + c55*c75 + c56*c76 + 1/10 - 1/4
    eq14 = c53*c83 + c54*c84 + c55*c85 + c56*c86 + 1/10 - 1/4
    eq15 = c53*c93 + c54*c94 + c55*c95 + c56*c96 + 1/10 - 1/4
    eq16 = c63*c73 + c64*c74 + c65*c75 + c66*c76 + 1/10 - 1/4
    eq17 = c63*c83 + c64*c84 + c65*c85 + c66*c86 + 1/10 - 1/4
    eq18 = c63*c93 + c64*c94 + c65*c95 + c66*c96 + 1/10 - 1/4
    eq19 = c73*c83 + c74*c84 + c75*c85 + c76*c86 + 1/10 - 1/4
    eq20 = c73*c93 + c74*c94 + c75*c95 + c76*c96 + 1/10 - 1/4
    eq21 = c83*c93 + c84*c94 + c85*c95 + c86*c96 + 1/10 - 1/4

    return [eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8, eq9, eq10, eq11, eq12, eq13, eq14, eq15, eq16, eq17, eq18, eq19, eq20, eq21]

# Initial guess for the optimization
initial_guess = [0.1] * 21

# Solve the equations
solution = scipy.optimize.fsolve(equations, initial_guess)
print(solution)"""

[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1]


/var/folders/z5/8__h5b611fz3d61c954t7_540000gp/T/ipykernel_966/2247410375.py:6: RuntimeWarning: invalid value encountered in sqrt
  c36 = np.sqrt(1 - abs(c33)**2 - abs(c34)**2 - abs(c35)**2)
/var/folders/z5/8__h5b611fz3d61c954t7_540000gp/T/ipykernel_966/2247410375.py:7: RuntimeWarning: invalid value encountered in sqrt
  c46 = np.sqrt(1 - abs(c43)**2 - abs(c44)**2 - abs(c45)**2)
/var/folders/z5/8__h5b611fz3d61c954t7_540000gp/T/ipykernel_966/2247410375.py:8: RuntimeWarning: invalid value encountered in sqrt
  c56 = np.sqrt(1 - abs(c53)**2 - abs(c54)**2 - abs(c55)**2)
/var/folders/z5/8__h5b611fz3d61c954t7_540000gp/T/ipykernel_966/2247410375.py:9: RuntimeWarning: invalid value encountered in sqrt
  c66 = np.sqrt(1 - abs(c63)**2 - abs(c64)**2 - abs(c65)**2)
/var/folders/z5/8__h5b611fz3d61c954t7_540000gp/T/ipykernel_966/2247410375.py:10: RuntimeWarning: invalid value encountered in sqrt
  c76 = np.sqrt(1 - abs(c73)**2 - abs(c74)**2 - abs(c75)**2)
/var/folders/z5/8__h5b611fz3d61c954t7_540000

In [ ]:
def equations(vars):
    x, y = vars
    return [x**2 + y**2 + 2*(y)**(1)*x- 5, x +2*y -4 ]

def numerical_gradient(vars, h=1e-5):
    grad = []
    for i in range(len(vars)):
        vars_plus = vars.copy()
        vars_plus[i] += h
        vars_minus = vars.copy()
        vars_minus[i] -= h
        grad_i = (sum([eq**2 for eq in equations(vars_plus)]) - sum([eq**2 for eq in equations(vars_minus)])) / (2 * h)
        grad.append(grad_i)
    return grad

def gradient_descent(initial_guess, learning_rate, num_iterations):
    vars = initial_guess
    for _ in range(num_iterations):
        grad = numerical_gradient(vars)
        vars = [vars[i] - learning_rate * grad[i] for i in range(len(vars))]
    return vars

rand_guess = [-6, 5]
learning_rate = 0.01
num_iterations = 1000

sol = gradient_descent(rand_guess, learning_rate, num_iterations)

print("Solution using Gradient Descent with numerical gradients:")
print(sol)


Solution using Gradient Descent with numerical gradients:
[-8.471896350436003, 6.2358441534921]


using the hybrid method of Lagrange multipliers and then numerically solving the equations works very good but sensitvie to initial guess, also 
only does real numbers not complex


In [5]:
import numpy as np
from scipy.optimize import fsolve

delta = [.5, .5, .5, .5, .5, .5, .5]


# Define the function for the system of equations
def system_of_equations(vars):
    # Unpacking variables, C63 to C99 and y3 to y9
    C63, C64, C65, C66, C67, C68, C69, \
    C73, C74, C75, C76, C77, C78, C79, \
    C83, C84, C85, C86, C87, C88, C89, \
    C93, C94, C95, C96, C97, C98, C99, \
    y3, y4, y5, y6, y7, y8, y9 = vars
    global delta
    delta_33, delta_44, delta_55, delta_66, delta_77, delta_88, delta_99 = delta

    # Full set of 28 equations for Cij terms, based on previous interactions
    equations = [
        # Derivatives with respect to C63 to C69
        np.conj(C63)*y3,
        np.conj(C63) + np.conj(C64)*y4,
        np.conj(C63) + np.conj(C64) + np.conj(C65)*y5,
        np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66)*y6,
        np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67)*y7,
        np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67) + np.conj(C68)*y8,
        np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67) + np.conj(C68) + np.conj(C69)*y9,
        # Continue with C73 to C79
        
        # Derivatives with respect to C73 to C79
        np.conj(C73)*y3,
        np.conj(C73) + np.conj(C74)*y4,
        np.conj(C73) + np.conj(C74) + np.conj(C75)*y5,
        np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76)*y6,
        np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77)*y7,
        np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77) + np.conj(C78)*y8,
        np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77) + np.conj(C78) + np.conj(C79)*y9,

        # Derivatives with respect to C83 to C89
        np.conj(C83)*y3,
        np.conj(C83) + np.conj(C84)*y4,
        np.conj(C83) + np.conj(C84) + np.conj(C85)*y5,
        np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86)*y6,
        np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87)*y7,
        np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87) + np.conj(C88)*y8,
        np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87) + np.conj(C88) + np.conj(C89)*y9,
        # Derivatives with respect to C93 to C99
        np.conj(C93)*y3,
        np.conj(C93) + np.conj(C94)*y4,
        np.conj(C93) + np.conj(C94) + np.conj(C95)*y5,
        np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96)*y6,
        np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97)*y7,
        np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97) + np.conj(C98)*y8,
        np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97) + np.conj(C98) + np.conj(C99)*y9,
        #Now the normalization equations
        abs(C63)**2 + abs(C73)**2 + abs(C83)**2 + abs(C93)**2 + delta_33 - 1,
        abs(C64)**2 + abs(C74)**2 + abs(C84)**2 + abs(C94)**2 + delta_44 - 1,
        abs(C65)**2 + abs(C75)**2 + abs(C85)**2 + abs(C95)**2 + delta_55 - 1,
        abs(C66)**2 + abs(C76)**2 + abs(C86)**2 + abs(C96)**2 + delta_66 - 1,
        abs(C67)**2 + abs(C77)**2 + abs(C87)**2 + abs(C97)**2 + delta_77 - 1,
        abs(C68)**2 + abs(C78)**2 + abs(C88)**2 + abs(C98)**2 + delta_88 - 1,
        abs(C69)**2 + abs(C79)**2 + abs(C89)**2 + abs(C99)**2 + delta_99 - 1
    ]

    # Example placeholder for constraints based on previous description
    # equations.extend([
    #     # Example for y3
    #     y3 - (np.abs(C63)**2 + ...),  # Complete this based on your specific constraint
    #     # Continue with y4 to y9
    # ])
    # Ensure there are 35 equations in total
    assert len(equations) == 35
    return equations

# Initial guess for the variables (Cij real and imaginary parts, y)
# initial_guess = [0.1] * 35  
initial_guess = [0.2] * 15 + [-.2]*15 + [0.1] * 5  

# Solve the system of equations
solution = fsolve(system_of_equations, initial_guess)

# Print the solution
print("Solution to the system:", solution)

# residues sum
residuals = system_of_equations(solution)
residuals_sum = np.sum(np.abs(residuals))
print("Residuals:", residuals)
print("Residuals sum:", residuals_sum)

Solution to the system: [ 5.39947499e-02 -5.39947499e-02  3.41479678e-01 -3.41479678e-01
 -3.49382845e-01  3.49382845e-01  1.36024980e-01  5.43783422e-02
 -5.43783422e-02  3.37911719e-01 -3.37911719e-01 -3.59205510e-01
  3.59205510e-01  1.45687225e-01  7.02075490e-01 -7.02075490e-01
  2.27088003e-01 -2.27088003e-01 -4.98689480e-01  4.98689480e-01
  6.68160881e-01 -3.48936782e-02  3.48936782e-02 -4.66517243e-01
  4.66517243e-01  1.45544506e-02 -1.45544506e-02 -1.17615790e-01
  2.65383007e-13  1.00000000e+00 -8.95312658e-13  1.00000000e+00
  3.72621111e-13  1.00000000e+00  2.02656113e-11]
Residuals: [1.4329289114494227e-14, 8.89843754237063e-14, -2.3941606956446403e-13, -1.4241885448740277e-11, -3.5830365414834474e-12, -5.655476087440547e-13, -3.3992242188665557e-12, 1.443108797144431e-14, 9.413303470040546e-14, -2.312395045494266e-13, -1.4496293054833131e-11, -3.953847428263887e-12, -6.227240945122503e-13, -3.41774148409909e-12, 1.8631890489990362e-13, -4.3787196091216174e-13, -9.359509

In [6]:
import sympy as sym
from sympy import pprint
import math as m
import numpy as np
import scipy.optimize
from sympy import pprint
from scipy.optimize import fsolve
from scipy.optimize import least_squares
from scipy.optimize import minimize


w = m.e**((2/3)*m.pi*(1j))     # third root of unity
POVM_unnormalized = np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]])  # unnormalized POVM direction vectors
POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # normalized POVM direction vectors


# defining -perf a function that checks normalization of the vector in numpy or sympy
def normalize(vector):
    return np.sum(np.abs(vector)**2)

# Another way, Creating a dictionary of pairs of Vector numbers and their inner product
# POVM_vec_np = (1/(2**0.5)) * np.array(POVM_unnormalized)
POVM_vec_np = np.array(POVM_unnormalized)
def qm_inner_product(vec1, vec2):
    return np.vdot(vec1, vec2)
inner_products = {}             # Compute the inner products and store in a dictionary
for i in range(len(POVM_vec_np)):
    for j in range(len(POVM_vec_np)):
        key = (i+1, j+1)
        value = qm_inner_product(POVM_vec_np[i], POVM_vec_np[j])
        if value.imag < 1e-14: 
            value = value.real 
            # value = round(value, 8)
        inner_products[key] = value 
# for key, value in inner_products.items():
#     print(f"Inner product of vectors {key}: {value}")
# verified it is symmertric, normalization holds etc.


#now finding the fifth element each by using the orthogonality condition of vector number 2 (which is known) and vectors j (number 3 and after)

c4j_list = [2,.5,.5,.5, (-.25-.433013j), -.25, .5, -.25, (-.25-.433013j)]           # fourth elements found earlier, not normalized at this point
# extending the 3d povm to 4d now that the fourth elements are known. this will be used for inner products and subsequently to find the fifth element.
four_lists = []                         # creating a list of the first four elements of each vector.
for i in range(9):
    # four_list = POVM_unnormalized[i].append(c4j_list[i])
    four_list = np.append(POVM_unnormalized[i], c4j_list[i])
    four_lists.append(four_list)


# storing possible pairs' inner products
four_inner_products = {}             
for i in range(len(four_lists)):
    for j in range(len(four_lists)):
        key = (i+1, j+1)
        value = np.vdot(four_lists[i], four_lists[j])
        if value.imag < 1e-14: 
            value = value.real 
            # value = round(value, 8)
        four_inner_products[key] = value

c5j_list = []                   # storing the fifth element for each vector after computing it.
for j in range(1,10):
    pair = (2,j)
    # print(f'four-inner product value associated to four_vectors: {pair} : ', four_inner_products[pair])
    c5j = -(2/np.sqrt(15))*four_inner_products[pair]           # see notebook
    # print(f'c5{pair[1]}:', c5j)
    c5j_list.append(c5j)

# print('list of 5th entries (c52 is wrong, should be sqrt(15)/2): ',c5j_list)        # second entry is wrong here, careful

#correct list, Second entry corrected from -1.1618 to root(15)/2 (see notebook)
# c5j_list = [-0.0, 1.936492 , 0.38729833462074165, (-0.38729833462074176-0.44721359549995776j), -0.19364916731037082, 0.5809475019311124, -0.38729833462074165, (-0.19364916731037093-0.4472135954999579j), 0.5809475019311124]
c5j_list = [-0.0, (15)**(.5)/2 , 0.38729833462074165, (-0.38729833462074176-0.44721359549995776j), -0.19364916731037082, 0.5809475019311124, -0.38729833462074165, (-0.19364916731037093-0.4472135954999579j), 0.5809475019311124]


# creating List of first five entries, Used for creating full vector later
five_lists = []                         
for i in range(9):
    five_list = np.append(four_lists[i], c5j_list[i])
    five_lists.append(five_list)

# inner products of the 5d vectors, for normalization 
five_inner_products = {}
for i in range(len(five_lists)):
    key = (i+1, i+1)
    value = np.vdot(five_lists[i], five_lists[i])
    if value.imag < 1e-14: 
        value = value.real 
        # value = round(value, 12)
    five_inner_products[key] = value


# print('Inner products of 5d vectors: ', five_inner_products)         


symbols = sym.symbols('c61, c71, c81, c91, c62, c72, c82, c92, c63, c73, c83, c93, c64, c74, c84, c94, c65, c75, c85, c95, c66, c76, c86, c96, c67, c77, c87, c97, c68, c78, c88, c98, c69, c79, c89, c99')


# Create the full 10D vectors v_i. Context: there are 9 free variables, we are setting the 9th enttry to be identically equal to 0 (tenth one is already zero). 
# There is another, probably better, method of cleverly using these 9 variables to actually reduce the number of equations. In that case, we use these 9 variables in the first 2 vectors, 5 and 4 
# respectively and then using these vectors we eliminate further coefficients or the unknown variables leaving us with actually only 21 equations(or something like that) .
NineDimVs_initial = []
for i in range(9):
    known_part = five_lists[i]            # Numerical entries from the five lists
    abstract_part = symbols[i*4:(i+1)*4]                         #  symbolic entries
    vector =  (1/sym.sqrt(6))*sym.Matrix(np.append(known_part, abstract_part)) 
    # NineDimVs.append(vector)
    NineDimVs_initial.append(vector.evalf())

NineDimVs_initial[0] = NineDimVs_initial[0].subs({'c61':0,'c71':0,'c81':0,'c91':0})     #manually changing the first vector and the second vector (The unknowns were all set to zero).
NineDimVs_initial[1] = NineDimVs_initial[1].subs({'c62':0,'c72':0,'c82':0,'c92':0})

u1_4, u2_4, u3_4, u4_4, u5_4, u6_4, u7_4, u8_4, u9_4 = sym.Matrix(NineDimVs_initial[0]),sym.Matrix(NineDimVs_initial[1]),sym.Matrix(NineDimVs_initial[2]),sym.Matrix(NineDimVs_initial[3]),sym.Matrix(NineDimVs_initial[4]),sym.Matrix(NineDimVs_initial[5]),sym.Matrix(NineDimVs_initial[6]),sym.Matrix(NineDimVs_initial[7]),sym.Matrix(NineDimVs_initial[8])
#above u vectors are the normalized vectors. the '_4' is Due to the fact that there are four unknown entries in each of those vectors (except the first two vectors which are fully known).

# pprint(np.sqrt(6)*u3_4) #eg (visualizing the unnormalized one, coz they are easier to understand)
# pprint(sym.sqrt(6)*u4)


# creating equations now amongst the last seven equations, all possible pairs, giving total 21 equations
# Not forgetting to take conjugates while doing inner product.

# Assuming NineDimVecs is a list of NumPy arrays, if not, convert them first
# For example, if originally in SymPy: u1 = np.array([complex(sym.re(u1), sym.im(u1)) for u1 in sympy_vector])

# List of nine vectors
# NineDimVecs = [u1, u2, u3, u4, u5, u6, u7, u8, u9]                   # uncomment this line and comment the next one if using the last entry as normalization entry
NineDimVecs = [u1_4, u2_4, u3_4,u4_4,u5_4,u6_4,u7_4,u8_4,u9_4]      
Equations_ortho_dict = {}

# Create pairs of equations and store in dictionary
for i in range(len(NineDimVecs)):
    for j in range(i , len(NineDimVecs)):
        key = f'E_{i+1}{j+1}'  # Create key as 'E_ij'
        # Compute inner product using NumPy with conjugate
        expression = np.vdot(NineDimVecs[i], NineDimVecs[j])
        Equations_ortho_dict[key] = expression

# how to substitute values in the equations
# tt = Equations_ortho_dict['E_35']
# pprint(tt)
# # print('','\n \n \n \n \n \n ',)
# print(tt.subs({'c64':0,'c74':0,'c84':0, 'c63':0, 'c73':0, 'c83':0, 'c65': 0,'c75': 0, 'c85': 0}))
# print(Equations_ortho_dict)

# lambdifying for faster numerical evaluation. Below is lambdifyied dictionary of equations
## uncomment the below line and comment out the line afterwards in order to use the version with normalization entry
# equations_lambdas = {key: sym.lambdify(('c63', 'c73', 'c83', 'c64', 'c74', 'c84', 'c65', 'c75', 'c85', 'c66', 'c76', 'c86', 'c67', 'c77', 'c87', 'c68', 'c78', 'c88', 'c69', 'c79', 'c89'), eq, 'numpy') for key, eq in Equations_ortho_dict.items()}
equations_lambdas = {key: sym.lambdify(('c63', 'c73', 'c83', 'c93', 'c64', 'c74', 'c84', 'c94', 'c65', 'c75', 'c85', 'c95', 'c66', 'c76', 'c86', 'c96', 'c67', 'c77', 'c87', 'c97', 'c68', 'c78', 'c88', 'c98', 'c69', 'c79', 'c89', 'c99'), eq, 'numpy') for key, eq in Equations_ortho_dict.items()}

# equations_lambdas['E_34'](1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1)  # works

def Equations_ortho_obj_fn(vars):                                # vars is the list of variables (42 in total (56 if normalization entry is not in terms of other entries), first 21 are magnitudes, remaining 21 are corresponding phases)
    # if len(vars) < 42:  # in case less than 42 variables are given, we will pad it with zeros
    #     while len(vars) < 42:
    #         vars.append(0)
    
    # throw error if length of vars  is not even 
    if len(vars) % 2 != 0:
        raise ValueError('Number of variables must be even, check the input vars')

    magnitudes = vars[:int(len(vars)/2)]
    phases = vars[int(len(vars)/2):]
    complex_vars = [r * np.exp(1j * theta) for r, theta in zip(magnitudes, phases)]
    
    # Unpack complex variables as needed for your equations
    # c63, c73, c83, c64, c74, c84, c65, c75, c85, c66, c76, c86, c67, c77, c87, c68, c78, c88, c69, c79, c89 = complex_vars 
    c63, c73, c83, c93, c64, c74, c84, c94, c65, c75, c85, c95, c66, c76, c86, c96, c67, c77, c87, c97, c68, c78, c88, c98, c69, c79, c89, c99 = complex_vars 

    # Define or reference your equations here; this is a placeholder
    eqn_order = ['E_34', 'E_35', 'E_36', 'E_37', 'E_38', 'E_39', 'E_45', 'E_46', 'E_47', 'E_48', 'E_49', 'E_56', 'E_57', 'E_58', 'E_59', 'E_67', 'E_68', 'E_69', 'E_78', 'E_79', 'E_89']
    
    # Assuming Equations_ortho_dict is defined globally and contains the equations
    substituted_eqns = []
    for i in eqn_order:
        substituted_eqns.append(equations_lambdas[i](*complex_vars))
    
    residual_eqns = [abs(i)**2 for i in substituted_eqns]   # avoiding the problem of making both the real and imaginary parts zero individually. It works as long as the magnitudes/"residue" go to zero.
    output_residue = residual_eqns + residual_eqns  # for tricking the fsolve function to not throw errror "input length must match the output number of equations..". Doing this simply doubles the total residue.
    # return output_residue                         # , uncomment it and comment next when using fsolve instead of minimize
    return sum(output_residue)










# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************
# **************************The solving part************************************


# Creating a list of delta_ii, ie. the inner products of the five vectors with themselves. To be used in the normalization equations in the solver.

delta_five = []           
# for i in range(9):
#     delta_five.append(np.vdot(five_lists[i], five_lists[i]))
# print(delta_five)     # each elt is real, as it should be
for i in range(2,9):
    vec_i = NineDimVecs[i]
    vec_i_5 = vec_i[:5]         # first 5 entries    
    delta_i_5 = normalize(vec_i_5)  
    delta_five.append(delta_i_5)

# Define the function for the system of equations
def system_of_equations(vars):
    # Unpacking variables, C63 to C99 and y3 to y9
    C63, C64, C65, C66, C67, C68, C69, \
    C73, C74, C75, C76, C77, C78, C79, \
    C83, C84, C85, C86, C87, C88, C89, \
    C93, C94, C95, C96, C97, C98, C99, \
    y3, y4, y5, y6, y7, y8, y9 = vars
    global delta_five
    # delta_11, delta_22, delta_33, delta_44, delta_55, delta_66, delta_77, delta_88, delta_99 = delta_five
    delta_33, delta_44, delta_55, delta_66, delta_77, delta_88, delta_99 = delta_five

    # Full set of 28 equations for Cij terms, based on previous interactions
    equations = [
        # Derivatives with respect to C63 to C69
        np.conj(C63)*y3,
        np.conj(C63) + np.conj(C64)*y4,
        np.conj(C63) + np.conj(C64) + np.conj(C65)*y5,
        np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66)*y6,
        np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67)*y7,
        np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67) + np.conj(C68)*y8,
        np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67) + np.conj(C68) + np.conj(C69)*y9,
        # Continue with C73 to C79
        
        # Derivatives with respect to C73 to C79
        np.conj(C73)*y3,
        np.conj(C73) + np.conj(C74)*y4,
        np.conj(C73) + np.conj(C74) + np.conj(C75)*y5,
        np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76)*y6,
        np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77)*y7,
        np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77) + np.conj(C78)*y8,
        np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77) + np.conj(C78) + np.conj(C79)*y9,

        # Derivatives with respect to C83 to C89
        np.conj(C83)*y3,
        np.conj(C83) + np.conj(C84)*y4,
        np.conj(C83) + np.conj(C84) + np.conj(C85)*y5,
        np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86)*y6,
        np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87)*y7,
        np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87) + np.conj(C88)*y8,
        np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87) + np.conj(C88) + np.conj(C89)*y9,
        # Derivatives with respect to C93 to C99
        np.conj(C93)*y3,
        np.conj(C93) + np.conj(C94)*y4,
        np.conj(C93) + np.conj(C94) + np.conj(C95)*y5,
        np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96)*y6,
        np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97)*y7,
        np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97) + np.conj(C98)*y8,
        np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97) + np.conj(C98) + np.conj(C99)*y9,
        #Now the normalization equations
        abs(C63)**2 + abs(C73)**2 + abs(C83)**2 + abs(C93)**2 + delta_33 - 1,
        abs(C64)**2 + abs(C74)**2 + abs(C84)**2 + abs(C94)**2 + delta_44 - 1,
        abs(C65)**2 + abs(C75)**2 + abs(C85)**2 + abs(C95)**2 + delta_55 - 1,
        abs(C66)**2 + abs(C76)**2 + abs(C86)**2 + abs(C96)**2 + delta_66 - 1,
        abs(C67)**2 + abs(C77)**2 + abs(C87)**2 + abs(C97)**2 + delta_77 - 1,
        abs(C68)**2 + abs(C78)**2 + abs(C88)**2 + abs(C98)**2 + delta_88 - 1,
        abs(C69)**2 + abs(C79)**2 + abs(C89)**2 + abs(C99)**2 + delta_99 - 1
        # normalize(NineDimVecs[2]) - 1,
        # normalize(NineDimVecs[3]) - 1,
        # normalize(NineDimVecs[4]) - 1,
        # normalize(NineDimVecs[5]) - 1,
        # normalize(NineDimVecs[6]) - 1,
        # normalize(NineDimVecs[7]) - 1,
        # normalize(NineDimVecs[8]) - 1
    ]
    # Ensure there are 35 equations in total
    assert len(equations) == 35
    return equations

# Initial guess for the variables (Cij real and imaginary parts, y)
# initial_guess = [0.1] * 35  

print("\n \n  delta five:", delta_five)

initial_guess = [0.2] * 15 + [-.2]*15 + [0.1] * 5  
print("Initial guess: \n ", initial_guess)
# Solve the system of equations

solution = fsolve(system_of_equations, initial_guess)
print("Solution to the system: \n", solution)


# residues sum
residuals = system_of_equations(solution)
residuals_sum = np.sum(np.abs(residuals))

print("Residuals:", residuals)
print("Residuals sum:", residuals_sum)





 
  delta five: [0.400000000000000, 0.433333333333333, 0.381250043028167, 0.400000000000000, 0.400000000000000, 0.383333333333333, 0.431250043028167]
Initial guess: 
  [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2, 0.1, 0.1, 0.1, 0.1, 0.1]
Solution to the system: 
 [ 2.59719166e-02 -2.52401661e-02  2.63746051e-02 -2.59719166e-02
 -2.59719166e-02  2.63301669e-02  2.52865199e-02  2.51626048e-02
 -2.44536563e-02  2.55527450e-02 -2.51626048e-02 -2.51626048e-02
  2.55096916e-02  2.44985657e-02  7.73654289e-01 -7.51856823e-01
  7.85649617e-01 -7.73654289e-01 -7.73654289e-01  7.84325886e-01
  7.53237617e-01  1.23022202e-02 -1.19556090e-02  1.24929633e-02
 -1.23022202e-02 -1.23022202e-02  1.24719140e-02  1.19775657e-02
 -3.98257779e-25  1.02899151e+00 -2.77445120e-02  1.04367945e+00
  4.36794495e-02  9.43308780e-01 -5.90310284e-02]
Residuals: [-1.0343517833657469e-26, -3.4694

In [7]:


# initial_guess = [0.2] * 15 + [-.2]*15 + [0.1] * 5  
# print("Initial guess: \n ", initial_guess)
# # Solve the system of equations

# solution = fsolve(system_of_equations, initial_guess)
# print("Solution to the system: \n", solution)


# # residues sum
# residuals = system_of_equations(solution)
# residuals_sum = np.sum(np.abs(residuals))

# print("Residuals:", residuals)
# print("Residuals sum:", residuals_sum)


# # More solutions from different initial guesses

# initial_guess = [0.1] * 35
# random initial guess, fixed seed
np.random.seed(43)
initial_guess = np.random.rand(35)  # random initial guess
print("Random initial guess (seeded): \n ", initial_guess)

solution = fsolve(system_of_equations, initial_guess)
print("Solution to the system: \n", solution)

# residues sum
residuals = system_of_equations(solution)
residuals_sum = np.sum(np.abs(residuals))

print("Residuals:", residuals)
print("Residuals sum:", residuals_sum)

# run above for 100 random initial guesses, pick residue solutions lesser than 1e-9.


for i in range(100):
    np.random.seed(i)
    initial_guess = np.random.rand(35)  # random initial guess
    solution = fsolve(system_of_equations, initial_guess)
    residuals = system_of_equations(solution)
    residuals_sum = np.sum(np.abs(residuals))
    if residuals_sum < 1e-9:
        print(f"initial_guess at seed {i}: \n {initial_guess}")
        print(f"Solution found for seed {i}: \n {solution}")
        #residues
        print("Residuals: \n ", residuals)
        print("Residuals sum:", residuals_sum)
    


Random initial guess (seeded): 
  [0.11505457 0.60906654 0.13339096 0.24058962 0.32713906 0.85913749
 0.66609021 0.54116221 0.02901382 0.7337483  0.39495002 0.80204712
 0.25442113 0.05688494 0.86664864 0.221029   0.40498945 0.31609647
 0.0766627  0.84322469 0.84893915 0.97146509 0.38537691 0.95448813
 0.44575836 0.66972465 0.08250005 0.89709858 0.2980035  0.26230482
 0.00512955 0.54320252 0.47559637 0.63637368 0.97820413]
Solution to the system: 
 [0.11505457 0.60906654 0.13339096 0.24058962 0.32713906 0.85913749
 0.66609021 0.54116221 0.02901382 0.7337483  0.39495002 0.80204712
 0.25442113 0.05688494 0.86664864 0.221029   0.40498945 0.31609647
 0.0766627  0.84322469 0.84893915 0.97146509 0.38537691 0.95448813
 0.44575836 0.66972465 0.08250005 0.89709858 0.2980035  0.26230482
 0.00512955 0.54320252 0.47559637 0.63637368 0.97820413]
Residuals: [0.034286663554381785, 0.2748156569248753, 0.7248053411648075, 0.9882009565707831, 1.2536878362705584, 1.971973230286267, 2.935950436704388, 0.16

In [1]:

# starting at 100 random initial points

seed_res_list = []

for i in range(1200):
    np.random.seed(i)
    initial_guess = np.random.rand(35)  # random initial guess
    solution = fsolve(system_of_equations, initial_guess)
    residuals = system_of_equations(solution)
    residuals_sum = np.sum(np.abs(residuals))
    if residuals_sum < 1e-11:
        #add to seed_res_list
        seed_res_list.append((residuals_sum, i))
    #sort the list by residue sum
    seed_res_list.sort()


seed_list = [i[1] for i in seed_res_list]   # create seed list for later use in class objects
print("Seed and residue sum list: \n", seed_res_list)


NameError: name 'np' is not defined

In [18]:
class seed_sol:
    def __init__(self, seed):
        self.seed = seed
    #defining the methods to find the solution, residuals and the sum of residuals
    def solution(self):
        solution = fsolve(system_of_equations, self.initial_guess)
        return solution
    def res_list(self):
        res_list = system_of_equations(self.solution())
        return res_list
    @property
    def initial_guess(self):
        np.random.seed(self.seed)
        initial_guess = np.random.rand(35)  # random initial guess
        return initial_guess
    @property
    def res(self):
        res = np.sum(np.abs(self.residuals))
        return res


In [19]:
sol_list = [seed_sol(i) for i in seed_list]  # create a list of class objects

In [37]:
for i in range(len(sol_list)):
    sol = sorted(abs((sol_list[i].solution()[-7:])))

[-1.583842570030145e-28, 0.012669918922296772, 0.0271809777002667, 0.0277445120456472, 0.9738963927917442, 0.9873300810777033, 1.028991510855053]
[-2.0235146889958515, -1.028991510855053, -0.9433087802858988, 1.591716399826981e-30, 0.043679449462603104, 0.956320550537397, 1.9417194121024022]
[-1.0289915108550531, -0.9563205505373968, -0.0590310284152023, 9.480232962436527e-23, 0.9433087802858989, 1.9417194121024024, 1.9563205505373968]
[-0.01266991892229635, -0.0011542979538714498, -2.384186655467141e-27, 0.02774451204564717, 0.9873300810777037, 0.9988914548725428, 1.028991510855053]


In [ ]:
# I have a bunch of coordinates in 9 dimensional space Write KNN algorithm to classify and visualize the points
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Sample data (replace this with your actual 9D data and labels)
data = np.random.rand(100, 9)  # 100 points in 9D space
labels = np.random.randint(0, 2, 100)  # Binary classification (0 or 1)

# Splitting the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Implementing KNN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Predicting the test set results
y_pred = knn.predict(X_test)

# Reducing dimensions for visualization using PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(data)

# Plotting the results
plt.figure(figsize=(10, 7))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='viridis', edgecolor='k', s=100)
plt.title("Visualization of 9D data in 2D using PCA")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.colorbar(label='Class Label')
plt.show()

